In [1]:
# Initialize and check health
from client import R2RBackend

client = R2RBackend()
response = client.health()
print(f'Health check response: {response}')

Health check response: ok


In [2]:
from stream_handler import R2RStreamHandler
stream_handler = R2RStreamHandler()

stream = client.prompt_llm(
    "How would you implement a script in python for secure file transfer using sockets and cryptographic primitives?",
    [{'role': 'assistant', 'content': 'You are a helpful assistant!'}],
    {'temperature': 0.5}
)

for chunk in stream_handler.process_stream(stream):
    print(chunk, end='', flush=True)

2024-11-02 21:21:33,374 - INFO - stream_handler - Processing chunk: <search>[{"extraction_id": "06f69d7d-92c1-5166-836c-6220c394f4ed", "document_id": "3dbfc547-0b09-51a5-add1-4a100f18d995", "user_id": "2acb499e-8428-543b-bd85-0d9098718220", "collection_ids": ["122fdf6a-e116-546b-a8f6-e4cb2e2c0a09"], "score": 0.7782077457657661, "text": "Figure 1-32. The OSI reference model.\n\nThe OSI model has seven layers. The principles that were applied to arrive at\n\nthe seven layers can be briefly summarized as follows:\n\n1. A layer should be created where a different abstraction is needed.\n\n2. Each layer should perform a well-defined function.\n\n3. The function of each layer should be chosen with an eye toward defining internationally standardized protocols.\n\n4. The layer boundaries should be chosen to minimize the information flow across the interfaces.\n\nSEC. 1.6", "metadata": {"version": "v0", "chunk_order": 238, "document_type": "pdf", "unstructured_filetype": "application/pdf", "uns

Based on the provided context, I will answer your question about the OSI model.The OSI (Open Systems Interconnection) model is a7-layered framework for designing and implementing network protocols. It was developed by the International Organization for Standardization (ISO) as a first step toward international standardization of the protocols used in computer networking.Here are the different layers of the OSI model, their functions, and the protocols associated with each layer:### Layer1: Physical Layer [Source: [10]]* Function: Defines the physical means of sending and receiving data between devices.* Protocol: Ethernet, Wi-Fi, etc.### Layer2: Data Link Layer [Source: [9], [10]]* Function: Ensures error-free transfer of data frames between two devices on the same network.* Sublayer:+ MAC (Media Access Control) sublayer [Source: [9]]- Function: Controls access to the physical medium and manages data transmission.- Protocol: Ethernet, Wi-Fi, etc.+ LLC (Logical Link Control) sublayer- F

In [ ]:
# File ingestion
from pathlib import Path
from utility.file_utils import iterate_over_files

folder_path = Path('data')
files = list(iterate_over_files(folder_path))
client.ingest_files(files) # Does not return anything. Logs can be traced to see which files were not ingested.

In [ ]:
# Chunks ingestion
urls = [
    "https://docs.streamlit.io/deploy/tutorials/docker#create-a-dockerfile", 
    "https://medium.com/@ypredofficial/rag-based-conversational-chatbot-using-streamlit-364c4c02c2f1", 
    "https://blog.gopenai.com/building-a-multi-pdf-rag-chatbot-langchain-streamlit-with-code-d21d0a1cf9e5",
    "https://medium.com/@pritubera/building-a-streamlit-chatbot-for-technical-questions-with-llamaindex-and-openai-26808c841dab",
    "https://r2r-docs.sciphi.ai/cookbooks/ingestion",
    "https://docs.streamlit.io/get-started/fundamentals/advanced-concepts",
    "https://de.wikipedia.org/wiki/Deutschland"
]

from utility.scraper import Scraper
from utility.splitter import Splitter

scraper = Scraper()
splitter = Splitter()

documents = scraper.fetch_documents(urls)
split_documents = splitter.split_documents(documents)

if split_documents is None:
    print("No chunks found for any URL!")
    exit()

# Go over all the urls.
for url in urls:
    chunks = [split_doc for split_doc in split_documents if split_doc.metadata['source'] == url]
    # If chunks are found for the corresponding url ingest them.
    if chunks:
        metadata = chunks[0].metadata
        chunks_text = [{"text": chunk.page_content} for chunk in chunks]
        try:
            resp = client.ingest_chunks(chunks_text, metadata)
            print(resp)
        except Exception as e:
            pass
    else:
        print(f"No chunks found for [{url}]!")
        continue 